# Graded Exercise Session: Circular Hough Transform

The circular Hough Transform can be used to detect circles in edge images. It is very
similar to the linear Hough Transform, with the difference that there is one parameter
more to take into account: circle radius.

Assume we are given a pair of an original gray-scale image ```I[y, x]``` and a corresponding
edge image `edge_I[y, x]`, of size $H \times W$, such that $0 ≤ y < H$ and $0 ≤ x < W$. Every
pixel of the edge image has logical value equal to 255 if the corresponding pixel of
the original image belongs to an edge, and 0 otherwise.

In this exercise you are asked to implement a version of Hough transform for detecting
circles. A circle can be represented in the following way:

\begin{equation*}
(x − x_c)^2 + (y − y_c)^2 = r^2, \hspace{0.5cm} 0 ≤ x_c < W, \hspace{0.5cm} 0 ≤ y_c < H, \hspace{0.5cm} \hspace{0.5cm} (1.1)
\end{equation*}

where $(y, x)$ are pixel coordinates in the edge image `edge_I`, and $(y_c, x_c, r)$ are the
parameters of the model corresponding to the y- and x- coordinates of the center of the circle and its radius respectively.
<p></p>
If the radius of the circle to be found is fixed, then the parameter space is reduced to
2-d. Each point $(y, x)$ on the original circle (in the image space) defines a circle centered
at $(y, x)$ with radius $r$. The intersection point of all such circles in the
parameter space corresponds to the center point $(y_c, x_c)$ of the original circle on the
image space. An example is shown below.

<img src="images/hough.png">
Figure 1. In order to find the center of the blue circle, green circles centered at four edge points vote for the blue circle center $(y_c, x_c)$.
<p></p>
An accumulator matrix is used to find the intersection point in the parameter space. For a fixed radius, the accumulator array is a 2-D matrix, whose
    rows and columns correspond to different values of parameters of $y_c$ and $x_c$ respectively. This matrix is filled by passing through every edge point, formulating a circle centered at that point in
the parameter space and increasing the voting number of all the cells/pixels through which the new
circle passes. The number of “votes” in each element in the accumulator matrix denotes the number of circles in 
the parameter space that pass through the corresponding cell. 
<p></p>
After voting, we compute local maxima of the accumulator array. The positions of these
maxima correspond to the circle centers $(y_c, x_c)$ in the original image space.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import math
from skimage.feature import peak_local_max
import cv2

### Q1. Creating Accumulator Array

* Implement a function ```accum = CircleHough(edge_I,r)``` as described below.

#### Inputs

- ```edge_I```: - edge image of size $H \times W $ with values 255 for edge pixels and 0 for non-edge pixels
- ```r```: - radius of the circle we want to detect

#### Outputs
- ```accum```: 2-d accumulator array of size $H \times W $ storing votes for every possible circle centre in the image

#### Proposed Steps.
<b>Note:</b> the proposed steps are only suggestion and you can implement the following functions however you wish to.

* Initialize the accumulator array with zeros.
* Find the set of coordinates $\{(y_e, x_e)\}$ of edge points in ```edge_I``` (tip: you can use function ```np.where(edge_I)``` )
* Implement a loop that does the following:
    * $\forall x \in \{x_e\}$ compute left and right boundaries $(x_l, x_r)$ of the circle to be investigated from $x$ and $r$
    * $\forall x_c \in [ x_l,x_r ]$ compute $y_{c1}, y_{c2}$ using Eq. 1.1
    * increment ```accum``` at positions $(y_{c1},x_c)$ and $(y_{c2},x_c)$ if they are within image boundaries and ignore them otherwise

    

In [ ]:
def CircleHough(edge_I,r):
    accum = np.zeros(edge_I.shape)
    edge_points = np.where(edge_I)
    for (i,j) in [(y,x) for y,x in zip(edge_points[0],edge_points[1])]:
        xmin = j-r
        xmax = j+r
        for xc in range(xmin,xmax+1):
            if (xc>=0 and xc<edge_I.shape[1]):
                yc1 = math.sqrt(r*r - (j-xc)*(j-xc))
                yc1 = -(yc1-i)
                yc2 = -math.sqrt(r*r - (j-xc)*(j-xc))
                yc2 = -(yc2-i)
                if (yc1>=0 and yc1<edge_I.shape[0]-1):
                    accum[int(yc1),xc] = accum[int(yc1),xc]+1
                if (yc2>=0 and yc2<edge_I.shape[0]-1):
                    accum[int(yc2),xc] = accum[int(yc2),xc]+1
    return accum    

* Below you can find the code to read the image and compute edges, you do not have to change it. Use ```edges``` image in the following exercises.

In [ ]:
img = cv2.imread('images/coins.png',0)
edges = cv2.Canny(img,200,400)
plt.imshow(img, cmap='gray')
plt.show()
plt.imshow(edges, cmap='magma')
plt.show()


* Compute and visualize accumulator arrays for the given ```edges``` image and $r_1 = 25$ and $r_2 = 30$. What is the visual difference between them and what is the reason for that? 

In [ ]:
accumulator_array_1 = CircleHough(edges,25)
accumulator_array_2 = CircleHough(edges,30)
plt.imshow(accumulator_array_1)
plt.figure()
plt.imshow(accumulator_array_2)
plt.show()

The accumulators detect circles of different radii - the first one detects smaller circles, while the second one larger. This is why the maxima in two differ.

### Q2. Detect Circles
Implement ```DetectCircles(edge_I,radii,thres)``` function as described below and compute the coordinates of the coin centers.

#### Input parameters
- ```edge_I```: The edge image computed using the Canny Edge Detector (already computed in the above section)
- ```radii```: List of radius values that you want to detect in the image. eg: radii = [21,23,25] 
- ```thres```: Threshold used for thresholding the local maxima in accumulator array. It's the minimum intensity of the peaks. 

#### Output
- The outputs are the two dictionaries ```y_c``` and ```x_c``` containing y- and x- coordinates of detected circles centers. See final step for more details. <b>Note</b> Instead of two dictionaries you can output also two arrays or two lists with corresponding radius.

#### Proposed Steps
Implement a loop such that for each radius value ```r``` in the radius range ```radii```, it does the following:
- Compute accumulator array for the edge image ```edge_I``` at radius ```r``` using function ```CircleHough```
- Find the local maxima that has peaks in the current accumulator array greater than the given threshold. You can use the function ```peak_local_max``` for this task. ***Hint***: you can use ```min_distance=20``` in the ```peak_local_max``` function.
- Store the x,y coordinates of the computed local maxima in x_c, y_c and return them as output from the function. ***Hint***: Use dictionaries for x_c and y_c.  
Sample code,
    * ```x_c = dict() #Initialize dictionary x_c``` 
    * ```x_c[r] = x_coordinates_of_centers_you_computed_at_radius_r # we are assigning a list of x-coordinates to x_c[r] ``` 

In [ ]:
def DetectCircles(edges,radii,thres):
    x_c = dict()
    y_c = dict()
    for r in radii:
        
        # Compute accum array for radius r
        accum = CircleHough(edges,r)
        
        # Compute local maxima locations
        yx = peak_local_max(accum, min_distance=20,threshold_abs=thres)

        # store the results in the output dictionary
        y_c[r] = yx[:,0]
        x_c[r] = yx[:,1]
        

    return y_c,x_c

radii = range(20,30);
thres = 48
y_c, x_c = DetectCircles(edges, radii,thres)

for r in y_c:
    print('\nradius- ' + str(r) + ': ',end='')
    for y,x in zip(y_c[r],x_c[r]):
        print(' (' + str(y) + ' ' + str(x) + ') ',end='')

### Q3. Visualization
To evaluate the approach we want to visualize the results. To this end implement a function ```ShowCircles(I,radii,x_c,y_c)``` as described below.
#### Inputs
- ```I```: original image
- ```radii```: a list of radii range
- ```x_c y_c ```: detected centers of circles

#### Proposed Steps
Implement a loop such that for each radius value ```r``` in the radius range ```radii``` does the following:
- Fetch the array of center points corresponding to the radius value ```r``` (e.g. ```x_c[r] and y_c[r]``` if you're using a dictionary).
- For each center point (cy, cx) in that array draw a circle with the corresponding radius ```r```
- Option 1: The set of points on the circle can be written as (cy+r∗sin(θ), cx+r∗cos(θ)). To reconstruct a circle iterate over theta values in range ```np.arange(0, 2*math.pi+1, 0.01)``` and for each point on the circle and the center point set the corresposing pixel value to white (255) to illustrate the circle. Functions ```math.sin()``` and ```math.cos()``` can be also useful.
- Option 2: If you are familiar with ```matplotlib```, you can create a figure using ```fig, ax = plt.subplots()``` and then you can define a circle using ```plt.Circle```. Finally you can add this circle to your figure using ```ax.add_patch()```. 

In [ ]:
def ShowCircles(I,radii,x_c,y_c):
    img = I.copy()
    fig,ax = plt.subplots(1)
    plt.axis('off')
    colormap='gist_rainbow'
    for r in radii:
        try:
            x_c_rad = x_c[r]
            y_c_rad = y_c[r]
            centers = list(zip(x_c_rad, y_c_rad))
            all_center_len = len(centers)
            for (cx, cy) in ([(x, y) for x,y in centers]):
                theta_range = np.arange(0, 2*math.pi+1, 0.01)
                for theta in theta_range:
                    circ_point_x = int(cx + r*math.cos(theta))
                    circ_point_y = int(cy + r*math.sin(theta))
                    img[circ_point_y, circ_point_x] = 255
                img[cy, cx] = 255
        except KeyError:
            continue
    plt.gray()
    ax.imshow(img)
    plt.show()

In [ ]:
def ShowCircles2(I,radii,x_c,y_c):
    img = I.copy()
    fig,ax = plt.subplots(1)
    plt.axis('off')
    colormap='gist_rainbow'
    for r in radii:
        try:
            x_c_rad = x_c[r]
            y_c_rad = y_c[r]
            centers = list(zip(x_c_rad, y_c_rad))
            all_center_len = len(centers)
            for (cx, cy) in ([(x, y) for x,y in centers]):
                color_circ = 'r'
                circ = Circle((cx, cy),r, color=color_circ, fill=False)
                ax.add_patch(circ)
                ax.plot([cx], [cy], 'ro',color=color_circ, mew=2, ms=4)
        except KeyError:
            continue
    plt.gray()
    ax.imshow(img)
    plt.show()

* Now apply your implementation to the image in Figure 1 (left). In Figure 1 (right) we show a
representative result of applying the circular Hough transform. The figure on the right is obtained by using ```matplotlib``` functions in Option 2. The illustration of the circles might change depending on your visualization code. <b>Note</b> If you did not manage to complete the previous exercises uncomment the two first lines in the next cell. They will load precomputed centers, so that you can check your answer.

<figure>
  <img src="images/coins.png" width="278" height="270" style="float: left;margin: 10px 90px 0px 90px; " />
    <img src="images/coins_result.png" width="328" height="328" style="float: center; margin: -20px 0px 0px 0px; "/>
   
<!--     <figcaption style="float: center; margin: 0px 0px 5px 200px;">
        </figcaption> -->
    Figure 1: Original image (left). Result of applying the circular Hough transform (right).
</figure>

In [ ]:
#Uncomment those lines if you did not manage to complete the previous exercises
#x_c  = np.load('x_center.npy').item()
#y_c = np.load('y_center.npy').item()
ShowCircles(img,radii,x_c,y_c)
ShowCircles2(img, radii, x_c, y_c)